### Collect maximum points in a matrix

Given an M x N matrix where each cell can have a value of 1, 0, or -1, where -1 denotes an unsafe cell, collect the maximum number of ones starting from the first cell and by visiting only safe cells (0 or 1). We can only go left or down if the row is odd; otherwise, we can only go right or down from the current cell.

Example:

    1  ->   1     -1     1      1
            
            |
              
    1  <-   0      0     -1     1
    
    |
    
    1  ->   1  ->  1  ->  1     -1
                          
                          |    
    
    -1      -1     1  <-  1      1
    
    
    1        1    -1     -1    1
    
    Maximum points collected is 9.
    


In [24]:
# with recursion

def collectPoints(matrix, row=0, col=0):
    if row < 0 or col < 0 or row >= len(matrix) or col >= len(matrix[0]) or matrix[row][col] == -1:
        return 0
    downPoints = collectPoints(matrix, row + 1, col)
    horizontalPoints = collectPoints(matrix, row, col+1) if row % 2 == 0 else collectPoints(matrix, row, col-1)
    return max(downPoints, horizontalPoints) + matrix[row][col]
    

In [25]:
matrix = [
    [1, 1, -1, 1, 1],
    [1, 0, 0, -1, 1],
    [1, 1, 1, 1, -1],
    [-1, -1, 1, 1, 1],
    [1, 1, -1, -1, 1]
]

In [55]:
%%time 
collectPoints(matrix)

CPU times: user 43 µs, sys: 1e+03 ns, total: 44 µs
Wall time: 47.2 µs


9

In [46]:
# with memoization

def collectPointsDy(matrix, row=0, col=0, lookup=None):
    if lookup == None:
        lookup = {}
    if row < 0 or col < 0 or row >= len(matrix) or col >= len(matrix[0]) or matrix[row][col] == -1:
        return 0
    key = f'{row}-{col}'
    if lookup.get(key):
        return lookup[key]
    downPoints = collectPointsDy(matrix, row+1, col, lookup)
    if row % 2 == 0:
        horizontalPoints = collectPointsDy(matrix, row, col+1, lookup)
    else:
        horizontalPoints = collectPointsDy(matrix, row, col-1, lookup)
    lookup[key] = max(downPoints, horizontalPoints) + matrix[row][col]
    return lookup[key]

In [53]:
%%time
collectPointsDy(matrix)

CPU times: user 36 µs, sys: 1 µs, total: 37 µs
Wall time: 40.1 µs


9

### Can also be solved in a bottom up approach, solving simpler problems first?

Example:
```
matrix = [
    [1, 1, -1, 1, 1],
    [1, 0, 0, -1, 1],
    [1, 1, 1, 1, -1],
    [-1, -1, 1, 1, 1],
    [1, 1, -1, -1, 1]
]
```

Create a lookup matrix and start at the entry point. If the value in the matrix is 1, +1 to the left cell in the lookup. If 0, carry the left cell over. If -1, we cannot continue on that row, so fill in the remaining cells with -1.

Next we have to move down and start from the right. If the top cell and right cell in the lookup is -1 (or no right cell), we cannot get to that cell, so fill it with a -1. If the matrix value is -1, fill in all remaining cells in the row with -1. Otherwise, take the max of the top and right and add the matrix value.

Outcome:
```
lookup = [
    [1, 2, -1, -1, -1],
    [3, 2, -1, -1, -1],
    [4, 5, 6, 7, -1],
    [-1, -1, 9, 8, -1],
    [-1, -1, -1, -1, -1]
```


In [41]:
def collectPointsBottomUp(matrix):
    lookup = [[0] * len(matrix[0]) for _ in range(len(matrix))]
    if matrix[0][0] == -1:
        return 0
    maxpoints = 0
    # establish the first row in the lookup
    lookup[0][0] = matrix[0][0]
    for col in range(1, len(matrix[0])):
        if matrix[0][col] == -1:
            for j in range(col, len(matrix[0])):
                lookup[0][j] = -1
            break
        else:
            lookup[0][col] = lookup[0][col-1] + matrix[0][col]
    for row in range(1, len(matrix)):
        for col in range(len(matrix[0])):
            # even rows we work left to right
            if row % 2 == 0:
                mval = matrix[row][col]
                # if cell is inaccessible or the matrix value is -1, fill in with -1
                if (lookup[row-1][col] == -1 and (col == 0 or lookup[row][col-1] == -1)) or mval == -1:
                    lookup[row][col] = -1
                else:
                    lookup[row][col] = max(lookup[row][col-1], lookup[row-1][col]) + mval if col > 0 else lookup[row-1][col] + mval
                    maxpoints = max(maxpoints, lookup[row][col])
            else:
                # odd rows we work from right to left
                colpos = len(matrix[0]) - 1 - col
                mval = matrix[row][colpos]
                # if the cell is inaccessible or the matrix value is -1, fill in with -1
                if (lookup[row - 1][colpos] == -1 and (col == 0 or lookup[row][colpos + 1] == -1)) or mval == -1:
                    lookup[row][colpos] = -1
                else:
                    lookup[row][colpos] = max(lookup[row - 1][colpos], lookup[row][colpos+1]) + mval if col > 0 else lookup[row-1][colpos] + mval
                    maxpoints = max(maxpoints, lookup[row][colpos])
    return maxpoints

In [59]:
%%time
collectPointsBottomUp(matrix)

CPU times: user 34 µs, sys: 1 µs, total: 35 µs
Wall time: 37.9 µs


9